In [151]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import math
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

In [152]:
df = pd.read_csv('C:\\Users\\DHAN RAJ\\Downloads\\btp_forecast_preprocessed_data\\delhi\\hourly\\imputed_hourly_delhi_data.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
data = df.drop(columns=['Timestamp'])
# Assuming df is your DataFrame
correlation_matrix = data.corr()
print(correlation_matrix)

                     PM2.5 (µg/m³)  PM10 (µg/m³)  NO (µg/m³)  NO2 (µg/m³)  \
PM2.5 (µg/m³)             1.000000      0.762668    0.365120     0.394483   
PM10 (µg/m³)              0.762668      1.000000    0.441521     0.441850   
NO (µg/m³)                0.365120      0.441521    1.000000     0.411800   
NO2 (µg/m³)               0.394483      0.441850    0.411800     1.000000   
NOx (ppb)                 0.424769      0.506704    0.936715     0.657088   
NH3 (µg/m³)               0.417735      0.445048    0.427931     0.418320   
SO2 (µg/m³)               0.202105      0.196856    0.146447     0.087885   
CO (mg/m³)                0.285521      0.417577    0.405056     0.221524   
Ozone (µg/m³)            -0.015239     -0.026832   -0.148200    -0.181012   
Benzene (µg/m³)           0.332918      0.416589    0.379373     0.316374   
Toluene (µg/m³)           0.315975      0.432868    0.463146     0.328473   
Eth-Benzene (µg/m³)       0.251269      0.372110    0.377531     0.232003   

In [170]:
# Set correlation threshold
threshold = 0.5

# Find pairs with correlation above threshold
highly_correlated_pairs = [(column1, 'PM2.5 (µg/m³)') for column1 in correlation_matrix.columns if abs(correlation_matrix[column1]['PM2.5 (µg/m³)']) > threshold and column1 != 'PM2.5 (µg/m³)']

In [171]:
from statsmodels.tsa.stattools import grangercausalitytests

# Apply Granger's Causality Test
for pair in highly_correlated_pairs:
    print()
    print(f"Granger Causality Test for pair {pair}:")
    granger_test_result = grangercausalitytests(df[list(pair)], maxlag=12)


Granger Causality Test for pair ('PM10 (µg/m³)', 'PM2.5 (µg/m³)'):

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=420.1599, p=0.0000  , df_denom=26276, df_num=1
ssr based chi2 test:   chi2=420.2079, p=0.0000  , df=1
likelihood ratio test: chi2=416.8836, p=0.0000  , df=1
parameter F test:         F=420.1599, p=0.0000  , df_denom=26276, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=198.5528, p=0.0000  , df_denom=26273, df_num=2
ssr based chi2 test:   chi2=397.1812, p=0.0000  , df=2
likelihood ratio test: chi2=394.2095, p=0.0000  , df=2
parameter F test:         F=198.5528, p=0.0000  , df_denom=26273, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=138.4267, p=0.0000  , df_denom=26270, df_num=3
ssr based chi2 test:   chi2=415.3909, p=0.0000  , df=3
likelihood ratio test: chi2=412.1418, p=0.0000  , df=3
parameter F test:         F=138.4267, p=0.0000  , df_denom=26270, df_num=3

Granger Causa

In [172]:
cols = [i[0] for i in highly_correlated_pairs]

In [174]:
cols.append('PM2.5 (µg/m³)')

In [175]:
cols

['PM10 (µg/m³)', 'PM2.5 (µg/m³)']

In [176]:
df_corr = df[cols]
df_corr.index = df['Timestamp']
from statsmodels.tsa.vector_ar.vecm import coint_johansen
johan_test_temp = df_corr
coint_johansen(johan_test_temp,-1,1).eig

array([0.09727122, 0.01007822])

In [177]:
colums = [i.split()[0] for i in cols]
val = pd.read_excel("C:/Users/DHAN RAJ/Downloads/delhi_hourly_data_validation.xlsx")
val['From Date'] = pd.to_datetime(val['From Date'])
val.set_index('From Date', inplace=True)
val = val[colums]
val.columns = cols
val

,PM10 (µg/m³),PM2.5 (µg/m³)
From Date,,
2024-01-01 00:00:00,353.65,192.07
2024-01-01 01:00:00,346.30,170.90
2024-01-01 02:00:00,352.10,155.45
2024-01-01 03:00:00,325.05,187.28
2024-01-01 04:00:00,298.98,127.58
2024-01-01 05:00:00,283.40,106.60
2024-01-01 06:00:00,268.12,115.92
2024-01-01 07:00:00,268.10,120.93
2024-01-01 08:00:00,267.12,121.00


In [178]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(df_corr)

# Normalizing values between 0 and 1
scaled_test = scaler.transform(val)

In [179]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=scaled_train)
model_fit = model.fit()

model_fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 27, Apr, 2024
Time:                     17:55:32
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -12.2919
Nobs:                     26279.0    HQIC:                  -12.2931
Log likelihood:           86962.9    FPE:                4.58036e-06
AIC:                     -12.2937    Det(Omega_mle):     4.57932e-06
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.021402         0.000720           29.716           0.000
L1.y1         0.846168         0.003961          213.623           0.000
L1.y2         0.149876         0.007312           20.498           0.000

Results for equation 

In [180]:
# make prediction on validation
lag_order = model_fit.k_ar
prediction = model_fit.forecast(scaled_train[-lag_order:], steps=24)

In [181]:
prediction

array([[0.37888762, 0.18513977],
       [0.3697527 , 0.17874955],
       [0.36106528, 0.17292784],
       [0.35284172, 0.16759552],
       [0.34508403, 0.16269061],
       [0.33778459, 0.15816376],
       [0.33092958, 0.1539749 ],
       [0.32450127, 0.15009093],
       [0.31847974, 0.14648403],
       [0.31284392, 0.14313037],
       [0.30757243, 0.14000927],
       [0.3026441 , 0.13710255],
       [0.29803825, 0.13439399],
       [0.29373498, 0.13186904],
       [0.28971526, 0.12951448],
       [0.28596102, 0.1273183 ],
       [0.28245514, 0.12526944],
       [0.2791815 , 0.12335776],
       [0.27612494, 0.12157388],
       [0.27327122, 0.11990911],
       [0.27060698, 0.11835541],
       [0.26811972, 0.11690529],
       [0.26579775, 0.1155518 ],
       [0.26363011, 0.11428846]])

In [182]:
scaled_test

array([[0.35216449, 0.19930748],
       [0.34475634, 0.17675261],
       [0.35060223, 0.16029192],
       [0.3233382 , 0.19420413],
       [0.29706194, 0.13059876],
       [0.28135867, 0.10824632],
       [0.26595777, 0.11817601],
       [0.26593761, 0.12351374],
       [0.26494986, 0.12358832],
       [0.25878143, 0.14146601],
       [0.2561407 , 0.12273599],
       [0.27594618, 0.11911357],
       [0.32708764, 0.15597699],
       [0.35523862, 0.19566375],
       [0.39754069, 0.20029832],
       [0.41438291, 0.14217984],
       [0.3906365 , 0.16574686],
       [0.3521141 , 0.14146601],
       [0.30200071, 0.14713403],
       [0.31423676, 0.17534626],
       [0.38212972, 0.23620286],
       [0.4261654 , 0.24099723],
       [0.44718037, 0.23345408],
       [0.4278083 , 0.21212444]])

In [183]:
val

,PM10 (µg/m³),PM2.5 (µg/m³)
From Date,,
2024-01-01 00:00:00,353.65,192.07
2024-01-01 01:00:00,346.30,170.90
2024-01-01 02:00:00,352.10,155.45
2024-01-01 03:00:00,325.05,187.28
2024-01-01 04:00:00,298.98,127.58
2024-01-01 05:00:00,283.40,106.60
2024-01-01 06:00:00,268.12,115.92
2024-01-01 07:00:00,268.10,120.93
2024-01-01 08:00:00,267.12,121.00


In [184]:
preds = scaler.inverse_transform(prediction)
preds

array([[380.16335364, 178.77218938],
       [371.10013763, 172.77433174],
       [362.48091348, 167.31007306],
       [354.32191744, 162.30515047],
       [346.62512284, 157.70140624],
       [339.38298573, 153.4525009 ],
       [332.58178119, 149.52083747],
       [326.20393928, 145.87534982],
       [320.22967116, 142.48990764],
       [314.63809243, 139.34216169],
       [309.40799062, 136.41270396],
       [304.51834169, 133.68445308],
       [299.94864963, 131.14220132],
       [295.67916205, 128.7722776 ],
       [291.69099924, 126.56229402],
       [287.96622324, 124.50095292],
       [284.48786565, 122.57789765],
       [281.23992759, 120.78359545],
       [278.20736109, 119.10924374],
       [275.37603846, 117.54669393],
       [272.73271434, 116.08838817],
       [270.26498359, 114.72730607],
       [267.96123715, 113.45691894],
       [265.81061747, 112.27114997]])

In [185]:
mae = np.abs(preds[:,1] - val['PM2.5 (µg/m³)']).mean()

In [186]:
mae

40.19947774969719

In [187]:
from statsmodels.tsa.vector_ar.vecm import VECM

# Fit VECM model
model = VECM(df_corr.values)
results = model.fit()

# Forecast
forecast = results.predict(steps=24)
mae = np.abs(forecast[:,len(cols)-1] - val['PM2.5 (µg/m³)']).mean()
mae

35.328913382019614